<a href="https://colab.research.google.com/github/park-hoyeon/park-hoyeon.github.io/blob/master/skt_7_07_OpenCV(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenCV Contours 처리


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
def show_image(title, img, cmap=None):
    """
    Helper function to display an image (BGR or Grayscale) using Matplotlib.
    - If the image has 3 channels, we assume it's BGR and convert to RGB.
    - If it's 2D, we assume a grayscale image.
    """
    plt.figure(figsize=(5,5))
    if len(img.shape) == 2:
        # Grayscale
        plt.imshow(img, cmap=cmap if cmap else 'gray')
    else:
        # BGR -> RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
    plt.title(title)
    plt.axis('off')
    plt.show()




In [ ]:
image = np.zeros((400,400), dtype=np.uint8)

cv2.rectangle(image, (50, 100), (300, 300), 255, -1)

contours, hierarchy = cv2.findContours(
    image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
)

print(f"Found {len(contours)} contours.")

output = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

cv2.drawContours(output, contours, -1, (0, 255, 255), 2)

show_image("Contours", output)

In [ ]:
# 1) Create a synthetic image (400x400, black background)
image = np.zeros((400, 400), dtype=np.uint8)  # single channel
# 2) Draw a white rectangle
cv2.rectangle(image, (50, 50), (350, 350), 255, -1)
# 3) Find contours
# Note: image must be an 8-bit single-channel binary (or thresholded) image.
contours, hierarchy = cv2.findContours(
    image.copy(),      # copy, to avoid modifying the original
    cv2.RETR_TREE,     # retrieve all contours in a tree structure
    cv2.CHAIN_APPROX_SIMPLE  # compress horizontal/vertical/diagonal segments
)
print(f"Number of contours found: {len(contours)}")
# 4) Draw contours on a BGR image for visualization
output = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)  # convert to 3-channel
cv2.drawContours(output, contours, -1, (0, 255, 0), 2)
show_image("Contours", output)

# 사각형 외곽

In [ ]:
contour = contours[0]

x, y, w, h = cv2.boundingRect(contour)
print(f"BoundingRect -> x:{x}, y:{y}, w:{w}, h:{h}")

output_rect = output.copy()

cv2.rectangle(output, (x, y), (x + w, y + h), (0, 0, 255), 2)

show_image("Bounding Rectangle", output_rect)

# Convex Hull

In [ ]:
hull = cv2.convexHull(contour)

output_hull = output.copy()

cv2.drawContours(output_hull, [hull], -1, (0, 0, 255), 2)

show_image("Convex Hull", output_hull)

# 유사 다각형 근사

In [ ]:
epsilon = 0.02 * cv2.arcLength(contour, True)
approx = cv2.approxPolyDP(contour, epsilon, True)

print("Number of points in original contour: ", len(contour))
print("Number of points in approximated contour: ", len(approx))

output_approx = output.copy()
cv2.drawContours(output_approx, [approx], -1, (255, 0, 255), 2)

show_image("Approximated Contour", output_approx)

# Contours


In [ ]:
area = cv2.contourArea(contour)
perimeter = cv2.arcLength(contour, True)
moments = cv2.moments(contour)

moments = cv2.moments(contour)
print(f"Moments: {moments}")
print("Contour Area:", area)
print("Contour Perimeter: ", perimeter)
4
if moments['m00'] != 0:
    cx = int(moments['m10'] / moments['m00'])
    cy = int(moments['m01'] / moments['m00'])
    print(f"Contour Center: (x={cx}, y={cy})")

In [ ]:
# Demonstrate minAreaRect
rotated_rect = cv2.minAreaRect(contour)  # ((cx, cy), (w, h), angle)
box_points = cv2.boxPoints(rotated_rect)  # 4 corner points
box_points = np.intp(box_points)
print("minAreaRect ->", rotated_rect)
print("Box points:", box_points)
output_minarea = output.copy()
cv2.drawContours(output_minarea, [box_points], -1, (255, 165, 0), 2)
show_image("minAreaRect Box", output_minarea)

# 과제


In [ ]:
# 다중 객체 이미지에서의 contours 처리 - 여러 개의 도형을 그린 이미지를 만들고 윤곽선을 찾아서 각각에 대한 bounding box, convex hull 시도하기.

image_multi = np.zeros((400, 400), dtype=np.uint8)

cv2.rectangle(image_multi, (50, 50), (150, 150), 255, -1)
cv2.circle(image_multi, (300, 100), 50, 255, -1)
triangle_pts = np.array([[200, 250], [350, 250], [275, 350]], np.int32)
cv2.fillPoly(image_multi, [triangle_pts], 255)

show_image("Multiple Shapes", image_multi)

contours_multi, hierarchy_multi = cv2.findContours(
    image_multi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
)

output_multi = cv2.cvtColor(image_multi, cv2.COLOR_GRAY2BGR)

cv2.drawContours(output_multi, contours_multi, -1, (0, 255, 255), 2)
show_image("Contours of 다중객체 이미지", output_multi)

output_multi_boxes_hulls = cv2.cvtColor(image_multi, cv2.COLOR_GRAY2BGR)

colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]

for i, contour in enumerate(contours_multi):
    # Bounding Box
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(output_multi_boxes_hulls, (x, y), (x + w, y + h), colors[i % len(colors)], 2)
    print(f"Contour {i+1}: BoundingRect -> x:{x}, y:{y}, w:{w}, h:{h}")

    # Convex Hull
    hull = cv2.convexHull(contour)
    cv2.drawContours(output_multi_boxes_hulls, [hull], -1, colors[i % len(colors)], 2)

show_image("Bounding Boxes and Convex Hulls", output_multi_boxes_hulls)


In [ ]:
#Contour 정렬 - 다중 윤곽선을 찾은 뒤, x좌표나 면적 등을 기준으로 정렬하여 나열하기

# 1) x 좌표를 기준으로 정렬
sorted_contours_by_x = sorted(contours_multi, key=lambda c: cv2.boundingRect(c)[0])

print("\nSorted contours by x-coordinate:")
output_sorted_x = cv2.cvtColor(image_multi, cv2.COLOR_GRAY2BGR)
for i, contour in enumerate(sorted_contours_by_x):

    M = cv2.moments(contour)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        cv2.putText(output_sorted_x, str(i+1), (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.drawContours(output_sorted_x, [contour], -1, colors[i % len(colors)], 2)

show_image("Contours Sorted by X-coordinate", output_sorted_x)


# 2) 면적을 기준으로 정렬 (큰 순서대로)
sorted_contours_by_area_desc = sorted(contours_multi, key=cv2.contourArea, reverse=True)

print("\nSorted contours by area (descending):")
output_sorted_area = cv2.cvtColor(image_multi, cv2.COLOR_GRAY2BGR)
for i, contour in enumerate(sorted_contours_by_area_desc):

    M = cv2.moments(contour)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        cv2.putText(output_sorted_area, str(i+1), (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.drawContours(output_sorted_area, [contour], -1, colors[i % len(colors)], 2)

show_image("Contours Sorted by Area (Descending)", output_sorted_area)

# 면적을 기준으로 정렬된 contours에 대해 bounding box와 convex hull 시도
print("\nBounding Boxes and Convex Hulls for Contours Sorted by Area:")
output_sorted_area_boxes_hulls = cv2.cvtColor(image_multi, cv2.COLOR_GRAY2BGR)

for i, contour in enumerate(sorted_contours_by_area_desc):
    # Bounding Box
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(output_sorted_area_boxes_hulls, (x, y), (x + w, y + h), colors[i % len(colors)], 2)
    print(f"Sorted Contour {i+1} (Area): BoundingRect -> x:{x}, y:{y}, w:{w}, h:{h}, Area: {cv2.contourArea(contour):.2f}")

    # Convex Hull
    hull = cv2.convexHull(contour)
    cv2.drawContours(output_sorted_area_boxes_hulls, [hull], -1, colors[i % len(colors)], 2)

show_image("Sorted by Area: Bounding Boxes and Convex Hulls", output_sorted_area_boxes_hulls)

In [ ]:
# 실제 이미지 사용 - "귀여움.jpg" 에서 Threshole, Canny 에지 검출 등을 적용해 contours를 찾고, 물체 외곽을 탐지하기

image_path = "귀여움.jpg"
try:
    image_real = cv2.imread(image_path)
    if image_real is None:
        raise FileNotFoundError(f"Image not found at {image_path}")

    show_image("Original Image", image_real)

    # Convert to grayscale
    gray_real = cv2.cvtColor(image_real, cv2.COLOR_BGR2GRAY)
    show_image("Grayscale Image", gray_real, cmap='gray')

    blur_real = cv2.GaussianBlur(gray_real, (5, 5), 0)
    show_image("Blurred Image", blur_real, cmap='gray')


    ret, thresh_real = cv2.threshold(blur_real, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    show_image("Thresholded Image (Otsu)", thresh_real, cmap='gray')

    show_image("Canny Edges", canny_real, cmap='gray')


    contours_thresh, hierarchy_thresh = cv2.findContours(
        thresh_real.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    print(f"Found {len(contours_thresh)} contours from thresholded image.")

    output_thresh_contours = image_real.copy()
    cv2.drawContours(output_thresh_contours, contours_thresh, -1, (0, 255, 255), 2)
    show_image("Contours from Threshold", output_thresh_contours)

    contours_canny, hierarchy_canny = cv2.findContours(
        canny_real.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    print(f"Found {len(contours_canny)} contours from Canny edges.")


    output_canny_contours = image_real.copy()
    cv2.drawContours(output_canny_contours, contours_canny, -1, (255, 0, 255), 2)
    show_image("Contours from Canny Edges", output_canny_contours)


    final_contours = contours_canny


    output_processed = image_real.copy()


    min_contour_area = 100
    large_contours = [c for c in final_contours if cv2.contourArea(c) > min_contour_area]
    print(f"Processing {len(large_contours)} large contours.")

    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255)] # More colors

    for i, contour in enumerate(large_contours):
        # Bounding Box
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(output_processed, (x, y), (x + w, y + h), colors[i % len(colors)], 2)
        print(f"Contour {i+1}: BoundingRect -> x:{x}, y:{y}, w:{w}, h:{h}, Area: {cv2.contourArea(contour):.2f}")

        # Convex Hull
        hull = cv2.convexHull(contour)
        cv2.drawContours(output_processed, [hull], -1, colors[i % len(colors)], 2) # Draw hull with the same color


    show_image("Processed Contours (Bounding Boxes and Convex Hulls)", output_processed)

except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Please make sure '귀여움.jpg' is uploaded to your Colab environment or mounted Drive.")
    print("You can use the 'files.upload()' function or mount Google Drive.")

except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow # Import cv2_imshow

img = cv2.imread("귀여움.jpg")
if img is None:
    print("이미지를 불러올 수 없습니다. 경로를 확인하세요.")
    # exit() # Avoid using exit() in Colab cells as it can stop the runtime

# 2. 그레이스케일 변환
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 3. 블러 처리 (노이즈 제거)
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# 4. 이진화 (Thresholding)
_, thresh = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)

# 5. Canny 에지 검출
edges = cv2.Canny(blur, 50, 150)

# 6. Contours 찾기 (에지 기준)
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Use edges.copy()

# 7. Contour 그리기 (원본 이미지에)
img_contours = img.copy()
cv2.drawContours(img_contours, contours, -1, (0, 255, 0), 2)

# 8. 결과 출력
cv2_imshow(img) # Use cv2_imshow
cv2_imshow(gray) # Use cv2_imshow
cv2_imshow(thresh) # Use cv2_imshow
cv2_imshow(edges) # Use cv2_imshow
cv2_imshow(img_contours) # Use cv2_imshow

# cv2.waitKey(0) # These are not typically needed with cv2_imshow in Colab
# cv2.destroyAllWindows() # These are not typically needed with cv2_imshow in Colab

In [ ]:
# 자세한 계층 구조 분석 - hierarchy를 이용해 부모-자식 윤곽선을 구분

img_hierarchy = cv2.imread("귀여움.jpg")
if img_hierarchy is None:
    print("이미지를 불러올 수 없습니다. 경로를 확인하세요.")
else:
    gray_hierarchy = cv2.cvtColor(img_hierarchy, cv2.COLOR_BGR2GRAY)

    # 임계값 적용
    ret_h, thresh_hierarchy = cv2.threshold(gray_hierarchy, 127, 255, cv2.THRESH_BINARY_INV) # 배경을 검정, 객체를 흰색으로

    # 외곽선과 계층 구조 찾기
    contours_h, hierarchy_h = cv2.findContours(
        thresh_hierarchy.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )

    # 결과 시각화를 위한 복사본 생성
    output_hierarchy = cv2.cvtColor(thresh_hierarchy, cv2.COLOR_GRAY2BGR)
    output_hierarchy_color = img_hierarchy.copy()

    print(f"\nFound {len(contours_h)} contours with hierarchy.")
    print("Hierarchy structure (next, previous, first_child, parent):")
    print(hierarchy_h)

    # 계층 구조를 사용하여 부모/자식 관계를 시각화합니다.
    # 초록색으로 부모, 파란색으로 자식 외곽선 그리기
    if hierarchy_h is not None:
        hierarchy_h = hierarchy_h[0]

        for i in range(len(contours_h)):
            parent_idx = hierarchy_h[i][3]

            if parent_idx == -1:
                # 부모가 없는 경우 (가장 바깥쪽 윤곽선) - 초록색
                cv2.drawContours(output_hierarchy_color, contours_h, i, (0, 255, 0), 2) # Green
            else:
                # 부모가 있는 경우 (내부 윤곽선) - 파란색
                cv2.drawContours(output_hierarchy_color, contours_h, i, (255, 0, 0), 2) # Blue

    show_image("Contours with Hierarchy (Green: Parent, Blue: Child)", output_hierarchy_color)



# 필터링


In [ ]:
!wget -O gray_image.jpg "https://blog.aspose.cloud/ko/imaging/grayscale-image-in-java/images/grayscale.jpg" -q
image_path = 'gray_image.jpg'  # 혹은 여러분의 이미지 파일명
image = cv2.imread(image_path)
if image is None:
    print("이미지를 불러오지 못했습니다. 경로를 확인하세요.")
else:
    print("이미지 로드 성공!")
    cv2_imshow(image)

# 과제
